In [ ]:
import os
import cv2
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, GlobalAveragePooling2D, Dropout
from keras.layers import Lambda, LSTM
from tensorflow.keras.optimizers import SGD
from keras.models import Model, load_model, Sequential
from keras.initializers import glorot_uniform
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

from keras.optimizers import adam_v2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

df1 = pd.read_csv("/kaggle/input/csv-creator-1/train.csv")
df1['image'] = 'csv-creator-1/' + df1['StudyInstanceUID'] + '-' + df1['SeriesInstanceUID'] + '-' + df1['SOPInstanceUID'] + '.png'

df2 = pd.read_csv("/kaggle/input/csv-creator-2/train.csv")
df2['image'] = 'csv-creator-2/' + df2['StudyInstanceUID'] + '-' + df2['SeriesInstanceUID'] + '-' + df2['SOPInstanceUID'] + '.png'

df3 = pd.read_csv("/kaggle/input/csv-creator-3/train.csv")
df3['image'] = 'csv-creator-3/' + df3['StudyInstanceUID'] + '-' + df3['SeriesInstanceUID'] + '-' + df3['SOPInstanceUID'] + '.png'

df4 = pd.read_csv("/kaggle/input/csv-creator-4/train.csv")
df4['image'] = 'csv-creator-4/' + df4['StudyInstanceUID'] + '-' + df4['SeriesInstanceUID'] + '-' + df4['SOPInstanceUID'] + '.png'

df5 = pd.read_csv("/kaggle/input/csv-creator-5/train.csv")
df5['image'] = 'csv-creator-5/' + df5['StudyInstanceUID'] + '-' + df5['SeriesInstanceUID'] + '-' + df5['SOPInstanceUID'] + '.png'

df6 = pd.read_csv("/kaggle/input/csv-creator-6/train.csv")
df6['image'] = 'csv-creator-6/' + df6['StudyInstanceUID'] + '-' + df6['SeriesInstanceUID'] + '-' + df6['SOPInstanceUID'] + '.png'

df7 = pd.read_csv("/kaggle/input/csv-creator-7/train.csv")
df7['image'] = 'csv-creator-7/' + df7['StudyInstanceUID'] + '-' + df7['SeriesInstanceUID'] + '-' + df7['SOPInstanceUID'] + '.png'

df = pd.concat([df1,df2,df3,df4,df5,df6,df7], ignore_index=True)
#df['image'] = df['image'][:12].replace('csv_creator_', 'csv-creator-') + df['image'][12:] #forgot to place the initial paths for the image paths; addding it here :P

df.to_csv('train.csv')


train_df, test_df = train_test_split(df, test_size=0.1)

# augmentation parameters
# you can use preprocessing_function instead of rescale in all generators
# if you are using a pretrained network
train_augmentation_parameters = dict(
    rescale=1.0/255.0,
    rotation_range=5,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range = [0.8, 1.2],
    validation_split = 0.2
)

valid_augmentation_parameters = dict(
    rescale=1.0/255.0,
    validation_split = 0.2
)

test_augmentation_parameters = dict(
    rescale=1.0/255.0
)

#<<=================================================================================================================================================>>
# training parameters
BATCH_SIZE = 32
CLASS_MODE = 'raw'
COLOR_MODE = 'rgb'
TARGET_SIZE = (512, 512)
EPOCHS = 10
SEED = 1337

train_consts = {
    'seed': SEED,
    'batch_size': BATCH_SIZE,
    'class_mode': CLASS_MODE,
    'color_mode': COLOR_MODE,
    'target_size': TARGET_SIZE,  
    'subset': 'training',
    'validate_filenames':False
}

valid_consts = {
    'seed': SEED,
    'batch_size': BATCH_SIZE,
    'class_mode': CLASS_MODE,
    'color_mode': COLOR_MODE,
    'target_size': TARGET_SIZE, 
    'subset': 'validation',
    'validate_filenames':False
}

test_consts = {
    'batch_size': 32,  # should be 1 in testing
    'class_mode': CLASS_MODE,
    'color_mode': COLOR_MODE,
    'target_size': TARGET_SIZE,  # resize input images
    'shuffle': False,
    'validate_filenames':False
}

#<<=================================================================================================================================================>>

src = '/kaggle/input'
x = 'image' #independent variables for DCMDataFrameIterator
y = ['pe_present_on_image'] ##dependent variables for DCMDataFrameIterator
#y = ['pe_present_on_image','leftsided_pe','central_pe','rightsided_pe','chronic_pe','acute_and_chronic_pe']

# Using the training phase generators 
train_augmenter = ImageDataGenerator(**train_augmentation_parameters)
valid_augmenter = ImageDataGenerator(**valid_augmentation_parameters)
test_augmenter = ImageDataGenerator(**test_augmentation_parameters)


train_generator = train_augmenter.flow_from_dataframe(dataframe = train_df, directory=src, x_col=x, y_col=y, **train_consts)

valid_generator = valid_augmenter.flow_from_dataframe(dataframe = train_df, directory=src, x_col=x, y_col=y, **valid_consts)

test_generator = test_augmenter.flow_from_dataframe(dataframe = test_df, directory=src, x_col=x, y_col=y, **test_consts)

In [ ]:
def identity_block(X, f, filters, stage, block):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. We'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X



def convolutional_block(X, f, filters, stage, block, s = 2):
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    
    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X


def ResNet50_build(input_shape = (64, 64, 3), classes = 2):   
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL.
    X = AveragePooling2D((2, 2), name='avg_pool')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
def build_model(input_shape, classes):
    inputs = Input(input_shape)
    resnet = ResNet50(include_top=False, input_shape=input_shape) 
    x  = GlobalAveragePooling2D()(resnet(inputs))
    x = Dropout(0.5)(x)
    outputs = Dense(classes, activation='sigmoid')(x)

    return Model(inputs, outputs)

In [ ]:
ROWS = 512
COLS = 512
CHANNELS = 3
CLASSES = 1

#model = ResNet50_build_model(input_shape = (ROWS, COLS, CHANNELS), classes = CLASSES)
model = build_model((ROWS, COLS, CHANNELS), CLASSES)

model.compile(optimizer=adam_v2.Adam(learning_rate=0.0001, decay=0.00001), 
              loss='binary_crossentropy', 
              metrics=['binary_accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_binary_accuracy', 
                              factor=0.5, 
                              patience=2, 
                              verbose=1, 
                              mode='max', 
                              min_lr=0.000001)

checkpoint = ModelCheckpoint("best_model.h5", 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')

model.summary()

In [ ]:
history = model.fit(train_generator,
                    epochs=EPOCHS,
                    validation_data=valid_generator,
                    verbose=1, 
                    callbacks=[checkpoint, reduce_lr]
)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator), batch_size=1)
print('test loss: {},  test_accuracy: {}'.format(test_loss, test_accuracy))

In [ ]:
model.save('model.h5')
model.save_weights('weights.h5')

In [ ]:
#loss graph
plt.plot(history.history['loss'],label='train loss')
plt.plot(history.history['val_loss'],label='val loss')
plt.legend()

plt.savefig('loss-graph.png')
plt.show()

# accuracies
plt.plot(history.history['binary_accuracy'], label='train acc')
plt.plot(history.history['val_binary_accuracy'], label='val acc')
plt.legend()

 
plt.savefig('acc-graph.png')
plt.show()